In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model.ndarray
%aimport allel.model.chunked
%aimport allel.model.dask
allel.__version__

'1.0.0b4.dev0'

## GenotypeVector

In [2]:
g = allel.GenotypeVector([[0, 1], [-1, -1]]*20)
g.is_phased = np.array([True, False]*20, dtype=bool)
g

0,1,2,3,4,...,35,36,37,38,39
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [3]:
g.display(30, 10)

0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,...,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [4]:
g.displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [5]:
print(repr(g))

<GenotypeVector shape=(40, 2) dtype=int64>
0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [6]:
print(g)

0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [7]:
print(g.to_str(40))

0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./.


In [8]:
g.concatenate(g)

0,1,2,3,4,...,75,76,77,78,79
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [9]:
g.concatenate(g, axis=1)

array([[ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1]])

## GenotypeArray

In [10]:
g = allel.GenotypeArray([[[0, 1], [-1, -1]]*20]*20)
g.is_phased = np.array([[True, False, False, False]*10]*20, dtype=bool)
g.mask = np.array([[1, 0, 0, 0, 0]*8]*20, dtype=bool)

In [11]:
print(g)

.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [12]:
print(repr(g))

<GenotypeArray shape=(20, 40, 2) dtype=int64>
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [13]:
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [14]:
g.display(row_threshold=10, row_edgeitems=5, col_threshold=20, col_edgeitems=10)

<GenotypeArray shape=(20, 40, 2) dtype=int64> 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39 0 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 1 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 2 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 3 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 4 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. ... ... 15 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 16 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 17 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 18 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 19 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./.

In [15]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [16]:
g[0]

0,1,2,3,4,...,35,36,37,38,39
.|.,./.,0/1,./.,0|1,...,./.,0|1,./.,0/1,./.


In [17]:
g[:, 4]

0,1,2,3,4,...,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,...,0|1,0|1,0|1,0|1,0|1


In [18]:
g[:, 4].displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1


In [19]:
g.mask = None
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [20]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [21]:
g.concatenate(g)

<GenotypeArray shape=(40, 40, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [22]:
g.concatenate(g, axis=1)

<GenotypeArray shape=(20, 80, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [23]:
g.to_n_alt()

array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 

## HaplotypeArray

In [24]:
h = g.to_haplotypes()

In [25]:
h

<HaplotypeArray shape=(20, 80) dtype=int64>
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
 ... 
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .

In [26]:
print(repr(h))

<HaplotypeArray shape=(20, 80) dtype=int64>
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
 ... 
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .



In [27]:
print(h)

0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
 ... 
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .



In [28]:
h.display(20, 40)

,0,1,2,3,4,...,75,76,77,78,79
0,0,1,.,.,0,...,.,0,1,.,.
1,0,1,.,.,0,...,.,0,1,.,.
2,0,1,.,.,0,...,.,0,1,.,.
3,0,1,.,.,0,...,.,0,1,.,.
4,0,1,.,.,0,...,.,0,1,.,.
5,0,1,.,.,0,...,.,0,1,.,.
6,0,1,.,.,0,...,.,0,1,.,.
7,0,1,.,.,0,...,.,0,1,.,.
8,0,1,.,.,0,...,.,0,1,.,.
9,0,1,.,.,0,...,.,0,1,.,.


In [29]:
h.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
1,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
2,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
3,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
4,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
5,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
6,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
7,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
8,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
9,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.


In [30]:
h.concatenate(h)

<HaplotypeArray shape=(40, 80) dtype=int64>
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
 ... 
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .

In [31]:
h.concatenate(h, axis=1)

<HaplotypeArray shape=(20, 160) dtype=int64>
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
 ... 
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .
0 1 . . 0  ...  . 0 1 . .

## AlleleCountsArray

In [32]:
ac = g.count_alleles()
ac

<AlleleCountsArray shape=(20, 2) dtype=int32>
20 20
20 20
20 20
 ... 
20 20
20 20
20 20

In [33]:
ac.displayall()

,0,1
0,20,20
1,20,20
2,20,20
3,20,20
4,20,20
5,20,20
6,20,20
7,20,20
8,20,20
9,20,20


In [34]:
print(ac)

20 20
20 20
20 20
 ... 
20 20
20 20
20 20



In [35]:
print(repr(ac))

<AlleleCountsArray shape=(20, 2) dtype=int32>
20 20
20 20
20 20
 ... 
20 20
20 20
20 20



In [36]:
ac + 5

<AlleleCountsArray shape=(20, 2) dtype=int32>
25 25
25 25
25 25
 ... 
25 25
25 25
25 25

In [37]:
ac + ac

<AlleleCountsArray shape=(20, 2) dtype=int32>
40 40
40 40
40 40
 ... 
40 40
40 40
40 40

In [38]:
ac.concatenate(ac)

<AlleleCountsArray shape=(40, 2) dtype=int32>
20 20
20 20
20 20
 ... 
20 20
20 20
20 20

In [39]:
ac.concatenate(ac, axis=1)

array([[20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20]], dtype=int32)

## GenotypeAlleleCounts

In [40]:
gac = g.to_allele_counts()
gac

<GenotypeAlleleCountsArray shape=(20, 40, 2) dtype=uint8>
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
 ... 
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0

In [41]:
print(gac)

1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
 ... 
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0



In [42]:
print(repr(gac))

<GenotypeAlleleCountsArray shape=(20, 40, 2) dtype=uint8>
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
 ... 
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1  ...  0:0 1:1 0:0 1:1 0:0



In [43]:
g[:, 0] = 2

In [44]:
g.to_allele_counts()

<GenotypeAlleleCountsArray shape=(20, 40, 3) dtype=uint8>
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
 ... 
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0

In [45]:
gac[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
1,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
2,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
3,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
4,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0


In [46]:
gac[0]

0,1,2,3,4,...,35,36,37,38,39
1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0


In [47]:
gac[:, 0]

0,1,2,3,4,...,15,16,17,18,19
1:1,1:1,1:1,1:1,1:1,...,1:1,1:1,1:1,1:1,1:1


In [48]:
gac[:3].is_called()

array([[ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False],
       [ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False],
       [ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False]], dtype=bool)

In [49]:
gac[0].is_called()

array([ True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False], dtype=bool)

## VariantTable

In [50]:
pos = np.unique(np.random.randint(0, 1000000, size=50000))
ref = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
alt = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
ra = np.rec.fromarrays([pos, ref, alt], names=['POS', 'REF', 'ALT'])
vtbl = allel.VariantTable(ra)
vtbl

<VariantTable shape=(48711,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(44, b'C', b'G') (65, b'A', b'C') (132, b'A', b'A') ...,
 (999929, b'C', b'C') (999965, b'G', b'T') (999996, b'C', b'A')]

In [51]:
vtbl[:5]

,POS,REF,ALT
0,44,b'C',b'G'
1,65,b'A',b'C'
2,132,b'A',b'A'
3,136,b'G',b'T'
4,159,b'T',b'T'


In [52]:
vtbl[:20].displayall()

,POS,REF,ALT
0,44,b'C',b'G'
1,65,b'A',b'C'
2,132,b'A',b'A'
3,136,b'G',b'T'
4,159,b'T',b'T'
5,224,b'C',b'T'
6,238,b'A',b'A'
7,261,b'T',b'C'
8,262,b'G',b'G'
9,263,b'T',b'C'


In [53]:
vtbl.take(list(range(5)))

,POS,REF,ALT
0,44,b'C',b'G'
1,65,b'A',b'C'
2,132,b'A',b'A'
3,136,b'G',b'T'
4,159,b'T',b'T'


In [54]:
vtbl.compress(np.random.randint(0, 2, size=vtbl.shape[0]).astype(bool))

<VariantTable shape=(24260,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(238, b'A', b'A') (261, b'T', b'C') (262, b'G', b'G') ...,
 (999757, b'C', b'G') (999889, b'A', b'C') (999965, b'G', b'T')]

In [55]:
vtbl.concatenate(vtbl)

<VariantTable shape=(97422,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(44, b'C', b'G') (65, b'A', b'C') (132, b'A', b'A') ...,
 (999929, b'C', b'C') (999965, b'G', b'T') (999996, b'C', b'A')]

In [56]:
vtbl[['POS', 'ALT']]

<VariantTable shape=(48711,) dtype=(numpy.record, [('POS', '<i8'), ('ALT', 'S1')])>
[(44, b'G') (65, b'C') (132, b'A') ..., (999929, b'C') (999965, b'T')
 (999996, b'A')]

In [57]:
print(vtbl)

[(44, b'C', b'G') (65, b'A', b'C') (132, b'A', b'A') ...,
 (999929, b'C', b'C') (999965, b'G', b'T') (999996, b'C', b'A')]


In [58]:
print(repr(vtbl))

<VariantTable shape=(48711,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(44, b'C', b'G') (65, b'A', b'C') (132, b'A', b'A') ...,
 (999929, b'C', b'C') (999965, b'G', b'T') (999996, b'C', b'A')]


## Chunked stuff

In [59]:
gc = allel.GenotypeChunkedArray(g).copy()
gc

<GenotypeChunkedArray shape=(20, 40, 2) dtype=int64 chunks=(20, 40, 2)
   nbytes=12.5K cbytes=450 cratio=28.4
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [60]:
print(gc)

2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
 ... 
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.



In [61]:
print(repr(gc))

<GenotypeChunkedArray shape=(20, 40, 2) dtype=int64 chunks=(20, 40, 2)
   nbytes=12.5K cbytes=450 cratio=28.4
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>


In [62]:
gc[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
3,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
4,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [63]:
gc[:, :5]

<GenotypeArray shape=(20, 5, 2) dtype=int64>
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
 ... 
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1

In [64]:
gc[5]

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [65]:
gc[:, 5]

0,1,2,3,4,...,15,16,17,18,19
./.,./.,./.,./.,./.,...,./.,./.,./.,./.,./.


In [66]:
x = gc.take([0, 2, 4], axis=0)
x

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [67]:
gc.take([0, 1, 2], axis=1)

<GenotypeChunkedArray shape=(20, 3, 2) dtype=int64 chunks=(20, 3, 2)
   nbytes=960 cbytes=386 cratio=2.5
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [68]:
gcna = gc.to_n_alt()
gcna

<ChunkedArrayWrapper shape=(20, 40) dtype=int8 chunks=(20, 40)
   nbytes=800 cbytes=360 cratio=2.2
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [69]:
gcna > 0

<ChunkedArrayWrapper shape=(20, 40) dtype=bool chunks=(20, 40)
   nbytes=800 cbytes=352 cratio=2.3
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [72]:
hc = gc.to_haplotypes()
hc

<HaplotypeChunkedArray shape=(20, 80) dtype=int64 chunks=(20, 80)
   nbytes=12.5K cbytes=428 cratio=29.9
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [73]:
hc.take([0, 1, 2], axis=1)

<HaplotypeChunkedArray shape=(20, 3) dtype=int64 chunks=(20, 3)
   nbytes=480 cbytes=361 cratio=1.3
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [74]:
ac = g.count_alleles()
ac

<AlleleCountsArray shape=(20, 3) dtype=int32>
19 19  2
19 19  2
19 19  2
 ... 
19 19  2
19 19  2
19 19  2

In [75]:
acc = gc.count_alleles()
acc

<AlleleCountsChunkedArray shape=(20, 3) dtype=int32 chunks=(20, 3)
   nbytes=240 cbytes=356 cratio=0.7
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [76]:
gc.is_called()

<ChunkedArrayWrapper shape=(20, 40) dtype=bool chunks=(20, 40)
   nbytes=800 cbytes=352 cratio=2.3
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [77]:
gc.to_n_alt()

<ChunkedArrayWrapper shape=(20, 40) dtype=int8 chunks=(20, 40)
   nbytes=800 cbytes=360 cratio=2.2
   compression=blosc compression_opts={'cname': 'lz4', 'shuffle': 1, 'clevel': 5}
   values=zarr.core.Array>

In [78]:
vctbl = allel.VariantChunkedTable(vtbl).copy()
vctbl

<VariantChunkedTable shape=(48711,) dtype=[('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')]
   nbytes=475.7K cbytes=130.5K cratio=3.6
   values=allel.chunked.storage_zarr.ZarrTable>

In [79]:
vctbl[:]

<VariantTable shape=(48711,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(44, b'C', b'G') (65, b'A', b'C') (132, b'A', b'A') ...,
 (999929, b'C', b'C') (999965, b'G', b'T') (999996, b'C', b'A')]

## Dask arrays

In [80]:
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
 ... 
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [81]:
gd = allel.GenotypeDaskArray(g)

In [82]:
gd.mask is None

True

In [83]:
gd.is_phased is None

True

In [84]:
print(repr(gd))

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>


In [85]:
print(gd)

2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
 ... 
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.



In [86]:
gd

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>

In [87]:
gd.values

dask.array<array-e..., shape=(20, 40, 2), dtype=int64, chunksize=(20, 40, 2)>

In [88]:
gd.values.name

'array-e1febcd166755510e9fdd2cd4926fe14'

In [89]:
gd[:]

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>

In [90]:
gd[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
3,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
4,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [91]:
gd.compute()

<GenotypeArray shape=(20, 40, 2) dtype=int64>
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
 ... 
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.

In [92]:
gd.concatenate(gd)

<GenotypeDaskArray shape=(40, 40, 2) dtype=int64>

In [93]:
gd.concatenate(gd, axis=1)

<GenotypeDaskArray shape=(20, 80, 2) dtype=int64>

In [94]:
hd = gd.to_haplotypes()
hd

<HaplotypeDaskArray shape=(20, 80) dtype=int64>

In [95]:
hd.compute()

<HaplotypeArray shape=(20, 80) dtype=int64>
2 2 . . 0  ...  . 0 1 . .
2 2 . . 0  ...  . 0 1 . .
2 2 . . 0  ...  . 0 1 . .
 ... 
2 2 . . 0  ...  . 0 1 . .
2 2 . . 0  ...  . 0 1 . .
2 2 . . 0  ...  . 0 1 . .

In [96]:
acd = gd.count_alleles()
acd

<AlleleCountsDaskArray shape=(20, 3) dtype=int64>

In [97]:
acd.compute()

<AlleleCountsArray shape=(20, 3) dtype=int64>
19 19  2
19 19  2
19 19  2
 ... 
19 19  2
19 19  2
19 19  2

In [98]:
gd[4]

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [99]:
gd[4].compute()

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [100]:
acd + acd

<AlleleCountsDaskArray shape=(20, 3) dtype=int64>

In [101]:
(acd + acd).compute()

<AlleleCountsArray shape=(20, 3) dtype=int64>
38 38  4
38 38  4
38 38  4
 ... 
38 38  4
38 38  4
38 38  4

In [102]:
gacd = allel.GenotypeAlleleCountsDaskArray(g.to_allele_counts())
gacd

<GenotypeAlleleCountsDaskArray shape=(20, 40, 3) dtype=uint8>

In [103]:
gacd.compute()

<GenotypeAlleleCountsArray shape=(20, 40, 3) dtype=uint8>
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
 ... 
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0  ...  0:0:0 1:1:0 0:0:0 1:1:0 0:0:0

In [104]:
gacd.count_alleles()

<AlleleCountsDaskArray shape=(20, 3) dtype=uint64>

In [105]:
gacd.count_alleles().compute()

<AlleleCountsArray shape=(20, 3) dtype=uint64>
19 19  2
19 19  2
19 19  2
 ... 
19 19  2
19 19  2
19 19  2